# Workshop Notebook 3: Automation with Wallaroo Connections

Wallaroo Connections are definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.

This provides MLOps engineers a method of creating and updating connection information for data stores:  databases, Kafka topics, etc.  Wallaroo Connections are composed of three main parts:

* Name:  The unique name of the connection.
* Type:  A user defined string that designates the type of connection.  This is used to organize connections.
* Details:  Details are a JSON object containing the information needed to make the connection.  This can include data sources, authentication tokens, etc.

Wallaroo Connections are only used to store the connection information used by other processes to create and use external connections.  The user still has to provide the libraries and other elements to actually make and use the conneciton.

The primary advantage is Wallaroo connections allow scripts and other code to retrieve the connection details directly from their Wallaroo instance, then refer to those connection details.  They don't need to know what those details actually - they can refer to them in their code to make their code more flexible.

For this step, we will use a Google BigQuery dataset to retrieve the inference information, predict the next month of sales, then store those predictions into another table.  This will use the Wallaroo Connection feature to create a Connection, assign it to our workspace, then perform our inferences by using the Connection details to connect to the BigQuery dataset and tables.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials Guide: Data Connections Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnections/)

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

import time
import pyarrow as pa

In [2]:
## convenience functions from the previous notebooks

# return the workspace called <name> through the Wallaroo client.
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
            return workspace
    # if no workspaces were found
    if workspace==None:
        raise KeyError(f"Workspace {name} was not found.")
    return workspace


# returns the most recent model version in a workspace for the matching `model_name`
def get_model_version(model_name, workspace):
    modellist = workspace.models()
    model_version = [m.versions()[-1] for m in modellist if m.name() == model_name]
    # if no models match, return None
    if len(modellist) <= 0:
        raise KeyError(f"Model {mname} not found in this workspace")
        return None
    return model_version[0]

# get a pipeline by name in the workspace
def get_pipeline(pipeline_name, workspace):
    plist = workspace.pipelines()
    pipeline = [p for p in plist if p.name() == pipeline_name]
    if len(pipeline) <= 0:
        raise KeyError(f"Pipeline {pipeline_name} not found in this workspace")
        return None
    return pipeline[0]


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
## blank space to log in 

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=THOD-BIPZ

Login successful!


### Set Configurations

Set the workspace, pipeline, and model used from Notebook 1.  The helper functions will make this task easier.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [4]:
## blank space to log in 

# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-john-forecast-demo"

workspace = get_workspace(workspace_name, wl)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# examine your current workspace
wl.get_current_workspace()

model_name = 'forecast-model'

prime_model_version = get_model_version(model_name, workspace)

pipeline_name = 'rental-forecast'

pipeline = get_pipeline(pipeline_name, workspace)

# display the workspace, model, and pipeline

display(wl.get_current_workspace())
display(pipeline)
display(prime_model_version)



{'name': 'workshop-workspace-john-forecast-demo', 'id': 22, 'archived': False, 'created_by': '1394d144-06a0-4b6b-b2db-d7945810e39c', 'created_at': '2023-11-13T17:06:07.813719+00:00', 'models': [{'name': 'forecast-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 11, 13, 17, 7, 26, 93495, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 11, 13, 17, 7, 26, 93495, tzinfo=tzutc())}, {'name': 'forecast-model-challenger-01', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 11, 13, 20, 42, 8, 51724, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 11, 13, 20, 42, 8, 51724, tzinfo=tzutc())}, {'name': 'forecast-model-challenger-02', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 11, 13, 20, 42, 9, 272465, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 11, 13, 20, 42, 9, 272465, tzinfo=tzutc())}], 'pipelines': [{'name': 'rental-forecast', 'create_time': datetime.datetime(2023, 11, 13,

name,rental-forecast
created,2023-11-13 17:07:28.018163+00:00
last_updated,2023-11-13 21:26:14.769759+00:00
deployed,False
arch,None
tags,
versions,"2ff4d42a-a9e3-4e2f-8136-f2905e78378f, 1525547a-7068-47ae-88b0-de0d67117998, ed1e809e-26a0-499e-93db-69cb87da0a8b, e97b3283-0856-4a89-89af-ba69e3a95e08, a3992425-bd73-4988-b42c-47c6e016288c, 7e43df46-2da3-4203-bf2a-ad16dfbf6ecc, 2a396c4c-45da-4620-92a0-a1b5973eb2c1, 94dd6ea2-ac9f-4477-ac6d-2972c8eefa93, 923e9499-ced4-4cda-af1d-1537d5352edc, 3430cf1b-1ece-4ca8-9b99-f71ed2faab32, 11ab670d-1eeb-4dd5-8f2f-d4e05f8bb403, 24a0f17d-8434-4da2-b784-b0a2bbad40f8, 3224a519-5caf-4de4-97bd-84fa95f9895a, 5cda6f54-cfde-4931-a873-4da4e141cea2, af62e7b2-8054-474d-a898-0384f1844e58, 3afd43f4-9d11-47c9-81da-2f65f0e916a0, 972c17d7-4524-4ae2-8ff8-683d3a8aee2a"
steps,forecast-model
published,False


Name,forecast-model
Version,a5eadca7-f3fb-411b-be16-82bc1f6835a5
File Name,forecast.py
SHA,a4b28a832f7aab63f8cbae411cf1c739bb08cf25685f466219fbd757a7ddffea
Status,ready
Image Path,None
Architecture,None
Updated At,2023-13-Nov 17:07:26


## Deploy the Pipeline with the Model Version Step

As per the other workshops:

1. Clear the pipeline of all steps.
1. Add the model version as a pipeline step.
1. Deploy the pipeline with the following deployment configuration:

```python
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
```

In [5]:
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,rental-forecast
created,2023-11-13 17:07:28.018163+00:00
last_updated,2023-11-13 22:08:36.400306+00:00
deployed,True
arch,None
tags,
versions,"52bcc0e1-8962-4ea1-9806-6fcdbb83351d, 2ff4d42a-a9e3-4e2f-8136-f2905e78378f, 1525547a-7068-47ae-88b0-de0d67117998, ed1e809e-26a0-499e-93db-69cb87da0a8b, e97b3283-0856-4a89-89af-ba69e3a95e08, a3992425-bd73-4988-b42c-47c6e016288c, 7e43df46-2da3-4203-bf2a-ad16dfbf6ecc, 2a396c4c-45da-4620-92a0-a1b5973eb2c1, 94dd6ea2-ac9f-4477-ac6d-2972c8eefa93, 923e9499-ced4-4cda-af1d-1537d5352edc, 3430cf1b-1ece-4ca8-9b99-f71ed2faab32, 11ab670d-1eeb-4dd5-8f2f-d4e05f8bb403, 24a0f17d-8434-4da2-b784-b0a2bbad40f8, 3224a519-5caf-4de4-97bd-84fa95f9895a, 5cda6f54-cfde-4931-a873-4da4e141cea2, af62e7b2-8054-474d-a898-0384f1844e58, 3afd43f4-9d11-47c9-81da-2f65f0e916a0, 972c17d7-4524-4ae2-8ff8-683d3a8aee2a"
steps,forecast-model
published,False


## Create the Connection

For this demonstration, the connection set to a specific file on a GitHub repository.  The connection details can be anything that can be stored in JSON:  connection URLs, tokens, etc.

This connection will set a URL to pull a file from GitHub, then use the file contents to perform an inference.

Wallaroo connections are created through the Wallaroo Client `create_connection(name, type, details)` method.  See the [Wallaroo SDK Essentials Guide: Data Connections Management guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnections/) for full details.

Note that connection names must be unique across the Wallaroo instance - if needed, use random characters at the end to make sure your connection doesn't have the same name as a previously created connection.

Here's an example connection used to retrieve the same CSV file used in `./data/testdata_standard.df.json`:  https://raw.githubusercontent.com/WallarooLabs/Workshops/main/Forecasting/Retail-CPG/data/testdata_standard.df.json

### Create the Connection Exercise

```python
# set the connection information for other steps
# suffix is used to create a unique data connection

forecast_connection_input_name = f'forecast-sample-data'
forecast_connection_input_type = "HTTP"
forecast_connection_input_argument = { 
    "url": "https://raw.githubusercontent.com/WallarooLabs/Workshops/main/Forecasting/Retail-CPG/data/testdata_standard.df.json"
    }


wl.create_connection(forecast_connection_input_name, forecast_connection_input_type, forecast_connection_input_argument)
```

In [12]:
wl.list_connections()

name,connection type,details,created at,linked workspaces
external_inference_connection,HTTP,*****,2023-11-01T18:51:51.359898+00:00,['simpleorchestrationworkspace']
forecast-sample-data,HTTP,*****,2023-11-13T22:08:59.048998+00:00,['workshop-workspace-john-forecast-demo']


In [13]:
# set the connection information for other steps
# suffix is used to create a unique data connection

forecast_connection_input_name = f'forecast-sample-data-total'
forecast_connection_input_type = "HTTP"
forecast_connection_input_argument = { 
    "url": "https://raw.githubusercontent.com/WallarooLabs/Workshops/20231113-forecast-updates/Forecasting/Retail-CPG/data/testdata-standard.df.json"
    }


wl.create_connection(forecast_connection_input_name, forecast_connection_input_type, forecast_connection_input_argument)

Field,Value
Name,forecast-sample-data-total
Connection Type,HTTP
Details,*****
Created At,2023-11-13T22:29:38.479076+00:00
Linked Workspaces,[]


## List Connections

Connections for the entire Wallaroo instance are listed with Wallaroo Client `list_connections()` method.

## List Connections Exercise

Here's an example of listing the connections when the Wallaroo client is `wl`.

```python
wl.list_connections()
```

In [7]:
# list the connections here

wl.list_connections()

name,connection type,details,created at,linked workspaces
external_inference_connection,HTTP,*****,2023-11-01T18:51:51.359898+00:00,['simpleorchestrationworkspace']
forecast-sample-data,HTTP,*****,2023-11-13T22:08:59.048998+00:00,[]


## Get Connection by Name

To retrieve a previosly created conneciton, we can assign it to a variable with the method Wallaroo `Client.get_connection(connection_name)`.  Then we can display the connection itself.  Notice that when displaying a connection, the `details` section will be hidden, but they are retrieved with `connection.details()`.  Here's an example:

```python
myconnection = client.get_connection("My amazing connection")
display(myconnection)
display(myconnection.details()
```

Use that code to retrieve your new connection.

### Get Connection by Name Example

Here's an example based on the Wallaroo client saved as `wl`.

```python
wl.get_connection(forecast_connection_input_name)
```

In [14]:
# get the connection by name

this_connection = wl.get_connection(forecast_connection_input_name)
this_connection

Field,Value
Name,forecast-sample-data-total
Connection Type,HTTP
Details,*****
Created At,2023-11-13T22:29:38.479076+00:00
Linked Workspaces,[]


## Add Connection to Workspace

We'll now add the connection to our workspace so it can be retrieved by other workspace users.  The method Workspace `add_connection(connection_name)` adds a Data Connection to a workspace.  The method Workspace `list_connections()` displays a list of connections attached to the workspace.

### Add Connection to Workspace Exercise

Use the connection we just created, and add it to the sample workspace.  Here's a code example where the workspace is saved to the variable `workspace` and the connection is saved as `forecast_connection_input_name`.

```python
workspace.add_connection(forecast_connection_input_name)
```

In [15]:
workspace.add_connection(forecast_connection_input_name)
workspace.list_connections()

name,connection type,details,created at,linked workspaces
forecast-sample-data,HTTP,*****,2023-11-13T22:08:59.048998+00:00,['workshop-workspace-john-forecast-demo']
forecast-sample-data-total,HTTP,*****,2023-11-13T22:29:38.479076+00:00,['workshop-workspace-john-forecast-demo']


## Retrieve Connection from Workspace

To simulate a data scientist's procedural flow, we'll now retrieve the connection from the workspace.  Specific connections are retrieved by specifying their position in the returned list.

For example, if we have two connections in a workspace and we want the second one, we can assign it to a variable with `list_connections[1]`.

Create a new variable and retrieve the connection we just assigned to the workspace.

### Retrieve Connection from Workspace Exercise

Retrieve the connection that was just associated with the workspace.  You'll use the `list_connections` method, then assign a variable to the connection.  Here's an example if the connection is the most recently one added to the workspace `workspace`.

```python
forecast_connection = workspace.list_connections()[-1]
```

In [17]:
forecast_connection = workspace.list_connections()[-1]
display(forecast_connection)

Field,Value
Name,forecast-sample-data-total
Connection Type,HTTP
Details,*****
Created At,2023-11-13T22:29:38.479076+00:00
Linked Workspaces,['workshop-workspace-john-forecast-demo']


## Run Inference with Connection

Connections can be used for different purposes:  uploading new models, engine configurations - any place that data is needed.  This exercise will use the data connection to perform an inference through our deployed pipeline.

### Run Inference with Connection Exercise

We'll now retrieve sample data through the Wallaroo connection, and perform a sample inference.  The connection details are retrieved through the Connection `details()` method.  Use them to retrieve the pandas record file and convert it to a DataFrame, and use it with our sample model.

Here's a code example that uses the Python `requests` library to retrieve the file information, then turns it into a DataFrame for the inference request.

```python
display(forecast_connection.details()['url'])

import requests

response = requests.get(
                    forecast_connection.details()['url']
                )

# display(response.json())

df = pd.DataFrame(response.json())

pipeline.infer(df)
```

In [18]:
display(forecast_connection.details()['url'])

import requests

response = requests.get(
                    forecast_connection.details()['url']
                )

# display(response.json())

df = pd.DataFrame(response.json())
display(df)

single_result = pipeline.infer(df)
display(single_result)

'https://raw.githubusercontent.com/WallarooLabs/Workshops/20231113-forecast-updates/Forecasting/Retail-CPG/data/testdata-standard.df.json'

,dteday,site_id,cnt,season,holiday,weekday,workingday
0,"[2011-02-02, 2011-02-03, 2011-02-04, 2011-02-05, 2011-02-06, 2011-02-07, 2011-02-08, 2011-02-09, 2011-02-10, 2011-02-11, 2011-02-12, 2011-02-13, 2011-02-14, 2011-02-15, 2011-02-16, 2011-02-17, 2011-02-18, 2011-02-19, 2011-02-20, 2011-02-21, 2011-02-22, 2011-02-23, 2011-02-24, 2011-02-25, 2011-02-26, 2011-02-27, 2011-02-28, 2011-03-01, 2011-03-02, 2011-03-03, 2011-03-04, 2011-03-05, 2011-03-06, 2011-03-07, 2011-03-08]","[site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001]","[1240, 1551, 2324, 805, 1948, 1650, 913, 931, 1256, 1614, 1000, 1883, 1964, 2036, 2586, 3219, 3947, 1826, 1418, 723, 1281, 2564, 2181, 1539, 2059, 2428, 836, 1235, -1, -1, -1, -1, -1, -1, -1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2]","[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]"


,time,in.cnt,in.dteday,in.holiday,in.season,in.site_id,in.weekday,in.workingday,out.dteday,out.forecast,out.site_id,check_failures
0,2023-11-13 22:30:25.767,"[1240, 1551, 2324, 805, 1948, 1650, 913, 931, 1256, 1614, 1000, 1883, 1964, 2036, 2586, 3219, 3947, 1826, 1418, 723, 1281, 2564, 2181, 1539, 2059, 2428, 836, 1235, -1, -1, -1, -1, -1, -1, -1]","[2011-02-02, 2011-02-03, 2011-02-04, 2011-02-05, 2011-02-06, 2011-02-07, 2011-02-08, 2011-02-09, 2011-02-10, 2011-02-11, 2011-02-12, 2011-02-13, 2011-02-14, 2011-02-15, 2011-02-16, 2011-02-17, 2011-02-18, 2011-02-19, 2011-02-20, 2011-02-21, 2011-02-22, 2011-02-23, 2011-02-24, 2011-02-25, 2011-02-26, 2011-02-27, 2011-02-28, 2011-03-01, 2011-03-02, 2011-03-03, 2011-03-04, 2011-03-05, 2011-03-06, 2011-03-07, 2011-03-08]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001, site0001]","[3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2]","[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]","[2011-03-02, 2011-03-03, 2011-03-04, 2011-03-05, 2011-03-06, 2011-03-07, 2011-03-08]","[2269, 1712, 1795, 1371, 1819, 2045, 1974]","[site0001, site0001, site0001, site0001, site0001, site0001, site0001]",0


## Cleaning up.

Now that the workshop is complete, don't forget to undeploy your pipeline to free up the resources.

In [19]:
pipeline.undeploy()

name,rental-forecast
created,2023-11-13 17:07:28.018163+00:00
last_updated,2023-11-13 22:08:36.400306+00:00
deployed,False
arch,None
tags,
versions,"52bcc0e1-8962-4ea1-9806-6fcdbb83351d, 2ff4d42a-a9e3-4e2f-8136-f2905e78378f, 1525547a-7068-47ae-88b0-de0d67117998, ed1e809e-26a0-499e-93db-69cb87da0a8b, e97b3283-0856-4a89-89af-ba69e3a95e08, a3992425-bd73-4988-b42c-47c6e016288c, 7e43df46-2da3-4203-bf2a-ad16dfbf6ecc, 2a396c4c-45da-4620-92a0-a1b5973eb2c1, 94dd6ea2-ac9f-4477-ac6d-2972c8eefa93, 923e9499-ced4-4cda-af1d-1537d5352edc, 3430cf1b-1ece-4ca8-9b99-f71ed2faab32, 11ab670d-1eeb-4dd5-8f2f-d4e05f8bb403, 24a0f17d-8434-4da2-b784-b0a2bbad40f8, 3224a519-5caf-4de4-97bd-84fa95f9895a, 5cda6f54-cfde-4931-a873-4da4e141cea2, af62e7b2-8054-474d-a898-0384f1844e58, 3afd43f4-9d11-47c9-81da-2f65f0e916a0, 972c17d7-4524-4ae2-8ff8-683d3a8aee2a"
steps,forecast-model
published,False


## Congratulations!

In this workshop you have:

* Deployed a single step house price prediction pipeline and sent data to it.
* Create a new Wallaroo connection.
* Assigned the connection to a workspace.
* Retrieved the connection from the workspace.
* Used the data connection to retrieve information from outside of Wallaroo, and use it for an inference.

Great job!